# Extracting GRO/PDB file from RTP

GROMACS natively have force-field parameters for small molecules included in rtp files. However, there are no coordinates of these molecules (PDB/GRO) included. Here is very simple algorithm to create these coordinates.
Sketch:
1. Extract atom names
2. Put atoms on a surface of sphere
3. Minimaze energy
4. Hope that there are no entanglements (like long alkane crossing in the middle of a ring)

Firstly, let's see what are avaible molecules in RTP file:

In [2]:
File = open("charmm36-mar2019.ff/merged.rtp")
text = File.read()
File.close()

In [3]:
all_residues = []
for line in text.splitlines():
    if len(line)>0:
        if line[0] == "[":
            all_residues.append(line.split()[1])
all_residues=all_residues[1:]

In [14]:
print("Number of residues:", len(all_residues))
print(all_residues[:20])

('Number of residues:', 1360)
['11BPO', '123TBB', '123TCB', '123TIB', '124TBB', '124TCB', '124TIB', '12A', '12DBB', '12DCB', '12DIB', '12MU', '135TBB', '135TCB', '135TIB', '13BPO', '13DB', '13DBB', '13DCB', '13DIB']


In CHARMM36 there are 1360 parametrized molecules. Let's select one of the residues, for example here we chose residue with name BENZ, which describes benzene:

In [16]:
selected = "BENZ"

Let's extract atom names from RTP file:

In [5]:
def atom_names(residue):
    inside = 0
    atom_names = [] 
    for line in text.splitlines():
        if inside>0:
            if "[" in line:
                inside+=1
            if inside==2 and "[" not in line:
                atom_names.append(line.split()[0])
        if residue in line:
            inside = 1
    return atom_names

Let's see what are the atoms in BENZ:

In [18]:
print(atom_names(selected))

['CG', 'HG', 'CD1', 'HD1', 'CD2', 'HD2', 'CE1', 'HE1', 'CE2', 'HE2', 'CZ', 'HZ']


Let's radomly sample points from a sphere:

In [7]:
import numpy as np

# Some things are faster to google then to implement:
# http://mathworld.wolfram.com/SpherePointPicking.html
# and the implementation:
# https://stackoverflow.com/questions/33976911/generate-a-random-sample-of-points-distributed-on-the-surface-of-a-unit-sphere

def sample_spherical():
    vec = np.random.randn(3, 1)
    vec /= np.linalg.norm(vec, axis=0)
    return 2*np.array(vec.T[0])

And add to every atom coordinante of radomly chosen place on a sphere:

In [8]:
def save_sphere_pdb(residue, atom_names):
    File = open(residue+".pdb","w")
    File.write("COMPND    {}\n".format(residue))
    File.write("AUTHOR    GENERATED\n")
    for atom in range(len(atom_names)):
        rand = sample_spherical()
        File.write("HETATM   {:2d}  {:3s} {}    0      {: 5.3f}  {: 5.3f}  {: 5.3f}  1.00  0.00           {}\n".format(atom, atom_names[atom],residue,rand[0],rand[1],rand[2], atom_names[atom][0]))
    File.write("END\n")
    File.close()

In [19]:
save_sphere_pdb(selected,atom_names(selected))

Now, let's visualze the results. To do that I use [nglview](https://github.com/arose/nglview) library:

In [20]:
import nglview
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [21]:
nglview.show_structure_file(selected+".pdb")

This is definitly questionable structure of benzene. But that's just start. Now we can perform energy minimalization and hope we obtain perfect structure. To do so, we will use [GROMACS](http://www.gromacs.org/) and python [GromacsWrapper](https://github.com/Becksteinlab/GromacsWrapper):

In [11]:
import gromacs

In [28]:
def generate_gro(residue, gro=None,v=True):
    if gro==None:
        gro=residue
    pdb_name = residue+".pdb"
    
    save_sphere_pdb(residue,atom_names(residue))
    # Generate topology for guessed PDB:
    gromacs.pdb2gmx(f=pdb_name, ff="charmm36-mar2019", water="none", o =residue+".gro", p =residue+".top")
    gromacs.editconf(f=residue+".gro",o=residue+".gro", box =[5,5,5])
    # Minimaze energy:
    gromacs.grompp(f="em.mdp", p=residue+".top",c=residue+".gro",o=gro+".tpr")
    gromacs.mdrun(deffnm=gro)
    # As return, let's visualize (if v true, otherwise let's don't do anything):
    if v:
        return nglview.show_structure_file(gro+".gro")
    else:
        return True

In [31]:
generate_gro(selected)

That's how you can obtain perfect GRO file. You have to remember that this method works for molecules without any rings and it MIGHT work for other molecules (but no promises) 

You can also easly convert it to PDB file by command:

In [25]:
gromacs.editconf(f=selected+".gro",o=selected+".pdb")

(0, None, None)

In [26]:
nglview.show_structure_file(selected+".pdb")

We can now create many coordinates files:

In [34]:
selected = ["MEOH","ETOH","PRO2","BENZ", "PHEN", "OCOH","BALD"]
for select in selected:
    generate_gro(select, v=False)